In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from os import path

import src.lib.utility_classfier_tuning as uclf
import src.lib.optimal_threhold_related as thres
import src.lib.fairness_tests as fair

from sklearn.metrics import balanced_accuracy_score, roc_auc_score
import sklearn.preprocessing
from sklearn import preprocessing

In [3]:
result_path='/Users/lifuchen/Desktop/Evaluating-and-Mitigating-Bias-in-ML-Models-for-CVD/external validation/'
filename = "gbt.csv"
gbt= pd.read_csv(path.join(result_path, filename))

In [4]:
gbt

,predict_score,true_label,GENDER,Race_B,Race_W
0,0.377786,0,0,0,1
1,0.103275,0,1,1,0
2,0.367792,0,1,0,1
3,0.384708,0,1,0,1
4,0.501749,0,1,0,1
...,...,...,...,...,...
8857,0.595026,1,0,0,1
8858,0.641625,1,1,0,1
8859,0.389285,1,1,1,0
8860,0.212954,1,0,1,0


In [5]:
y_test = gbt['true_label']
y_score = gbt['predict_score']

In [6]:
y_score_male = gbt['predict_score'][gbt['GENDER']==0]
y_test_male = gbt['true_label'][gbt['GENDER']==0]
y_score_female = gbt['predict_score'][gbt['GENDER']==1]
y_test_female = gbt['true_label'][gbt['GENDER']==1]
print(len(y_score_male), len(y_test_male), len(y_score_female), len(y_test_female))

3052 3052 5810 5810


In [7]:
y_score_white = gbt['predict_score'][gbt['Race_W']==1]
y_test_white = gbt['true_label'][gbt['Race_W']==1]
y_score_black = gbt['predict_score'][gbt['Race_W']==0]
y_test_black = gbt['true_label'][gbt['Race_W']==0]
print(len(y_score_white), len(y_score_black))

7583 1279


In [8]:
def get_result (records, y_test, y_score, y_test_1, y_score_1, y_test_2, y_score_2):    
    threshold = 0.1
    ba = thres.calculate_balanced_accuracy(y_test, y_score, threshold)
    auroc = roc_auc_score(y_test, y_score)
    precision, recall, tpr, tnr, pd_overall = thres.calculate_precision_metrics(y_test, y_score,threshold)
    
    ba_male = thres.calculate_balanced_accuracy(y_test_1, y_score_1, threshold)
    precision_male, recall_male, tpr_male, tnr_male, pd_male = thres.calculate_precision_metrics(y_test_1, y_score_1,threshold)
    
    ba_female = thres.calculate_balanced_accuracy (y_test_2, y_score_2, threshold)
    precision_female, recall_female, tpr_female, tnr_female, pd_female = thres.calculate_precision_metrics(y_test_2, y_score_2,threshold)

    eod = fair.get_EOD(y_test_1, y_score_1,threshold, y_test_2, y_score_2, threshold)
    sp = fair.get_SP(y_test_1, y_score_1,threshold, y_test_2, y_score_2, threshold)

    records.append({
        'auroc': auroc,
        'overall ba test': ba,
        'male ba test': ba_male,
        'female ba test': ba_female,
        'overall precision':precision,
        'overall recall':recall,
        'overall tpr':tpr,
        'overall tnr':tnr,
        'overall pd':pd_overall,
        'male precision':precision_male,
        'male recall':recall_male,
        'male tpr':tpr_male,
        'male tnr':tnr_male,
        'male pd':pd_male,
        'female precision':precision_female,
        'female recall':recall_female,
        'female tpr':tpr_female,
        'female tnr':tnr_female,
        'female pd':pd_female,
        'eod': eod,
        'di': sp,
        })

In [9]:
records_gbt = []
get_result(records_gbt, y_test, y_score, y_test_male, y_score_male, y_test_female, y_score_female)

True positive rate of class 1 is  0.997
True positive rate of class 2 is  0.985
Positive prediction rate of class 1 is  0.977
Positive prediction rate of class 2 is  0.963


In [10]:
records_gbt

[{'auroc': 0.6903449826345819,
  'overall ba test': 0.5121173406102704,
  'male ba test': 0.5109480122324159,
  'female ba test': 0.5120870257934457,
  'overall precision': 0.5061349693251533,
  'overall recall': 0.99,
  'overall tpr': 0.99,
  'overall tnr': 0.034,
  'overall pd': 0.968,
  'male precision': 0.505578093306288,
  'male recall': 0.997,
  'male tpr': 0.997,
  'male tnr': 0.025,
  'male pd': 0.977,
  'female precision': 0.5061664953751285,
  'female recall': 0.985,
  'female tpr': 0.985,
  'female tnr': 0.039,
  'female pd': 0.963,
  'eod': 0.01200000000000001,
  'di': 1.0145379023883696}]

In [11]:
records_gbt_race = []
get_result(records_gbt_race, y_test, y_score, y_test_white, y_score_white, y_test_black, y_score_black)
records_gbt_race

True positive rate of class 1 is  0.99
True positive rate of class 2 is  0.99
Positive prediction rate of class 1 is  0.97
Positive prediction rate of class 2 is  0.955


[{'auroc': 0.6903449826345819,
  'overall ba test': 0.5121173406102704,
  'male ba test': 0.5108558211806298,
  'female ba test': 0.5203397543560055,
  'overall precision': 0.5061349693251533,
  'overall recall': 0.99,
  'overall tpr': 0.99,
  'overall tnr': 0.034,
  'overall pd': 0.968,
  'male precision': 0.5056179775280899,
  'male recall': 0.99,
  'male tpr': 0.99,
  'male tnr': 0.032,
  'male pd': 0.97,
  'female precision': 0.5105724600309438,
  'female recall': 0.99,
  'female tpr': 0.99,
  'female tnr': 0.051,
  'female pd': 0.955,
  'eod': 0.0,
  'di': 1.0157068062827226}]